In [ ]:
## Refactoring SCTG Product Codes
import json
from neo4j import GraphDatabase

import urllib
import requests
import os
import sys
import time

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "testtest"))

wallclock = 0
label = ""
rel = ""
key = ""
values = []


query1 = """
CALL apoc.cypher.run('MATCH (o:MetroArea)-[s:SHIPMENT]-() WITH DISTINCT s.'+$key+' AS val ORDER BY val RETURN COLLECT(val) AS values',{}) 
YIELD value 
RETURN value.values AS values
"""


query2 = """
CALL apoc.cypher.run('MATCH (o:MetroArea)-[s:SHIPMENT {' +$key+ ':' +apoc.text.toCypher($val)+ '}]->(d:MetroArea) WHERE NOT (o)-[:' +$rel+ ']->(d) RETURN o,d,s',{}) YIELD value WITH value.o AS o, value.d AS d, value.s AS s 
WITH o,d,s, 
s.WGT_FACTOR * s.SHIPMT_VALUE AS value, 
s.WGT_FACTOR * (s.SHIPMT_WGHT*1.0)/2000 AS tonnage, 
s.WGT_FACTOR * (s.SHIPMT_WGHT*1.0)/2000 * s.SHIPMT_DIST_ROUTED AS ton_miles, 
s.WGT_FACTOR * s.SHIPMT_DIST_ROUTED AS wgt_miles, 
s.WGT_FACTOR AS factor, 
s.SHIPMT_DIST_ROUTED AS dist_routed, 
s.SHIPMT_DIST_GC AS dist_routed_gc 
WITH o,d, 
s.SCTG AS checkValue, 
COUNT(s) AS TOTAL_ANN_SHIPMENTS, 
SUM(factor) AS sum_wgt_factor, 
ROUND(SUM(value)) AS TOTAL_ANN_VALUE, 
ROUND(SUM(tonnage)) AS TOTAL_ANN_TONNAGE, 
ROUND(SUM(ton_miles)) AS TOTAL_ANN_TON_MILES, 
ROUND(SUM(wgt_miles)) AS TOTAL_ANN_WGT_MILES, 
ROUND(SUM(dist_routed)) AS TOTAL_ANN_MILES, 
ROUND(SUM(dist_routed_gc)) AS TOTAL_ANN_MILES_GC 
ORDER BY o,d
CALL {
    WITH o,d
    CALL apoc.create.relationship(o,$rel,{},d) 
    YIELD rel AS r 
    RETURN r 
}
WITH r, 
checkValue, 
TOTAL_ANN_SHIPMENTS, 
sum_wgt_factor, 
TOTAL_ANN_VALUE, 
TOTAL_ANN_TONNAGE, 
TOTAL_ANN_TON_MILES, 
TOTAL_ANN_WGT_MILES, 
TOTAL_ANN_MILES, 
TOTAL_ANN_MILES_GC 
SET r.SCTG = checkValue, 
r.TOTAL_ANN_SHIPMENTS = TOTAL_ANN_SHIPMENTS, 
r.TOTAL_ANN_VALUE = TOTAL_ANN_VALUE, 
r.TOTAL_ANN_TONNAGE = TOTAL_ANN_TONNAGE, 
r.TOTAL_ANN_TON_MILES = TOTAL_ANN_TON_MILES, 
r.TOTAL_ANN_WGT_MILES = TOTAL_ANN_WGT_MILES, 
r.TOTAL_ANN_MILES = TOTAL_ANN_MILES, 
r.TOTAL_ANN_MILES_GC = TOTAL_ANN_MILES_GC, 
r.TOTAL_AVG_MILES_PER_SHIPMENT = CASE WHEN sum_wgt_factor = 0 THEN 0 ELSE ROUND((TOTAL_ANN_WGT_MILES*1.0)/sum_wgt_factor) END
"""

database = "neo4j"

label = "Product"

key = "SCTG"

session = driver.session(database = database)

result = session.run(query1, label = label, key = key);
for record in result:
    values = record['values']
result.consume()

print(values)

for val in values:
    
    rel = val.replace('-','_') + '_2017_SHIPMENTS'
    
    print(rel)
    
    start_time = time.time()

    result = session.run(query2, key = key, val = val, rel = rel)
    result.consume()
    
    end_time = time.time()
    wallclock = round((end_time - start_time) *1000)
    
    print('Node: '+ str(rel) + ' completed in ' + str(wallclock))
    
print('done!')

driver.close()

In [ ]:
## Refactoring NAICS Codes
import json
from neo4j import GraphDatabase

import urllib
import requests
import os
import sys
import time

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "testtest"))

wallclock = 0
label = ""
rel = ""
key = ""
values = []


query1 = """
CALL apoc.cypher.run('MATCH (o:MetroArea)-[s:SHIPMENT]-() WITH DISTINCT s.'+$key+' AS val ORDER BY val RETURN COLLECT(val) AS values',{}) 
YIELD value 
RETURN value.values AS values
"""


query2 = """
CALL apoc.cypher.run('MATCH (o:MetroArea)-[s:SHIPMENT {' +$key+ ':' +apoc.text.toCypher($val)+ '}]->(d:MetroArea) WHERE NOT (o)-[:' +$rel+ ']->(d) RETURN o,d,s',{}) YIELD value WITH value.o AS o, value.d AS d, value.s AS s 
WITH o,d,s, 
s.WGT_FACTOR * s.SHIPMT_VALUE AS value, 
s.WGT_FACTOR * (s.SHIPMT_WGHT*1.0)/2000 AS tonnage, 
s.WGT_FACTOR * (s.SHIPMT_WGHT*1.0)/2000 * s.SHIPMT_DIST_ROUTED AS ton_miles, 
s.WGT_FACTOR * s.SHIPMT_DIST_ROUTED AS wgt_miles, 
s.WGT_FACTOR AS factor, 
s.SHIPMT_DIST_ROUTED AS dist_routed, 
s.SHIPMT_DIST_GC AS dist_routed_gc 
WITH o,d, 
s.NAICS AS checkValue, 
COUNT(s) AS TOTAL_ANN_SHIPMENTS, 
SUM(factor) AS sum_wgt_factor, 
ROUND(SUM(value)) AS TOTAL_ANN_VALUE, 
ROUND(SUM(tonnage)) AS TOTAL_ANN_TONNAGE, 
ROUND(SUM(ton_miles)) AS TOTAL_ANN_TON_MILES, 
ROUND(SUM(wgt_miles)) AS TOTAL_ANN_WGT_MILES, 
ROUND(SUM(dist_routed)) AS TOTAL_ANN_MILES, 
ROUND(SUM(dist_routed_gc)) AS TOTAL_ANN_MILES_GC 
ORDER BY o,d
CALL {
    WITH o,d
    CALL apoc.create.relationship(o,$rel,{},d) 
    YIELD rel AS r 
    RETURN r 
}
WITH r, 
checkValue, 
TOTAL_ANN_SHIPMENTS, 
sum_wgt_factor, 
TOTAL_ANN_VALUE, 
TOTAL_ANN_TONNAGE, 
TOTAL_ANN_TON_MILES, 
TOTAL_ANN_WGT_MILES, 
TOTAL_ANN_MILES, 
TOTAL_ANN_MILES_GC 
SET r.NAICS = checkValue, 
r.TOTAL_ANN_SHIPMENTS = TOTAL_ANN_SHIPMENTS, 
r.TOTAL_ANN_VALUE = TOTAL_ANN_VALUE, 
r.TOTAL_ANN_TONNAGE = TOTAL_ANN_TONNAGE, 
r.TOTAL_ANN_TON_MILES = TOTAL_ANN_TON_MILES, 
r.TOTAL_ANN_WGT_MILES = TOTAL_ANN_WGT_MILES, 
r.TOTAL_ANN_MILES = TOTAL_ANN_MILES, 
r.TOTAL_ANN_MILES_GC = TOTAL_ANN_MILES_GC, 
r.TOTAL_AVG_MILES_PER_SHIPMENT = CASE WHEN sum_wgt_factor = 0 THEN 0 ELSE ROUND((TOTAL_ANN_WGT_MILES*1.0)/sum_wgt_factor) END
"""

database = "neo4j"

label = "NAICS"

key = "NAICS"

session = driver.session(database = database)

result = session.run(query1, label = label, key = key);
for record in result:
    values = record['values']
result.consume()

print(values)

for val in values:
    
    rel = key + '_' + val + '_2017_SHIPMENTS'
    
    print(rel)
    
    start_time = time.time()

    result = session.run(query2, key = key, val = val, rel = rel)
    result.consume()
    
    end_time = time.time()
    wallclock = round((end_time - start_time) *1000)
    
    print('Node: '+ str(rel) + ' completed in ' + str(wallclock))
    
print('done!')

driver.close()

In [ ]:
## Refactoring HazMat by Rail and Water



import json
from neo4j import GraphDatabase

import urllib
import requests
import os
import sys
import time

uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "testtest"))

wallclock = 0
label = ""
rel = ""
key = ""
values = []


#MATCH (n:Mode) WHERE toLower(n.DESCRIPTION) CONTAINS 'rail' OR  toLower(n.DESCRIPTION) CONTAINS 'water' RETURN COLLECT(n.MODE) as values
#RETURN ["MODE_06", "MODE_07", "MODE_08", "MODE_101", "MODE_15", "MODE_16", "MODE_17"]
query1 = """
RETURN ["MODE_101"]
AS values
"""


query2 = """
CALL apoc.cypher.run('MATCH (o:MetroArea)-[s:SHIPMENT {' +$key+ ':' +apoc.text.toCypher($val)+ '}]->(d:MetroArea) WHERE NOT (o)-[:' +$rel+ ']->(d) RETURN o,d,s',{}) YIELD value WITH value.o AS o, value.d AS d, value.s AS s 
WHERE s.HAZMAT = 'P' OR s.HAZMAT = 'H'
WITH o,d,s, 
s.WGT_FACTOR * s.SHIPMT_VALUE AS value, 
s.WGT_FACTOR * (s.SHIPMT_WGHT*1.0)/2000 AS tonnage, 
s.WGT_FACTOR * (s.SHIPMT_WGHT*1.0)/2000 * s.SHIPMT_DIST_ROUTED AS ton_miles, 
s.WGT_FACTOR * s.SHIPMT_DIST_ROUTED AS wgt_miles, 
s.WGT_FACTOR AS factor, 
s.SHIPMT_DIST_ROUTED AS dist_routed, 
s.SHIPMT_DIST_GC AS dist_routed_gc 
WITH o,d, 
s.MODE AS checkValue, 
COUNT(s) AS TOTAL_ANN_SHIPMENTS, 
SUM(factor) AS sum_wgt_factor, 
ROUND(SUM(value)) AS TOTAL_ANN_VALUE, 
ROUND(SUM(tonnage)) AS TOTAL_ANN_TONNAGE, 
ROUND(SUM(ton_miles)) AS TOTAL_ANN_TON_MILES, 
ROUND(SUM(wgt_miles)) AS TOTAL_ANN_WGT_MILES, 
ROUND(SUM(dist_routed)) AS TOTAL_ANN_MILES, 
ROUND(SUM(dist_routed_gc)) AS TOTAL_ANN_MILES_GC 
ORDER BY o,d
CALL {
    WITH o,d
    CALL apoc.create.relationship(o,$rel,{},d) 
    YIELD rel AS r 
    RETURN r 
}
WITH r, 
checkValue, 
TOTAL_ANN_SHIPMENTS, 
sum_wgt_factor, 
TOTAL_ANN_VALUE, 
TOTAL_ANN_TONNAGE, 
TOTAL_ANN_TON_MILES, 
TOTAL_ANN_WGT_MILES, 
TOTAL_ANN_MILES, 
TOTAL_ANN_MILES_GC 
SET r.MODE = checkValue, 
r.TOTAL_ANN_SHIPMENTS = TOTAL_ANN_SHIPMENTS, 
r.TOTAL_ANN_VALUE = TOTAL_ANN_VALUE, 
r.TOTAL_ANN_TONNAGE = TOTAL_ANN_TONNAGE, 
r.TOTAL_ANN_TON_MILES = TOTAL_ANN_TON_MILES, 
r.TOTAL_ANN_WGT_MILES = TOTAL_ANN_WGT_MILES, 
r.TOTAL_ANN_MILES = TOTAL_ANN_MILES, 
r.TOTAL_ANN_MILES_GC = TOTAL_ANN_MILES_GC, 
r.TOTAL_AVG_MILES_PER_SHIPMENT = CASE WHEN sum_wgt_factor = 0 THEN 0 ELSE ROUND((TOTAL_ANN_WGT_MILES*1.0)/sum_wgt_factor) END
"""

database = "neo4j"

label = "Mode"

key = "MODE"

session = driver.session(database = database)

result = session.run(query1, label = label, key = key);
for record in result:
    values = record['values']
result.consume()

print(values)

for val in values:
    
    rel = 'HAZMAT_' + val + '_2017_SHIPMENTS'
    
    print(rel)
    
    start_time = time.time()

    result = session.run(query2, key = key, val = val, rel = rel)
    result.consume()
    
    end_time = time.time()
    wallclock = round((end_time - start_time) *1000)
    
    print('Node: '+ str(rel) + ' completed in ' + str(wallclock))
    
print('done!')

driver.close()